### This is a program to simulate tree based Pseudo-Least Recently Used(PLRU) replacement policy for an n-way set assoiative  cache

In [1]:
import math
import numpy as np

In [2]:
# create a node class for the bit tree that is maintained for each set
class Node(object):  
    def __init__(self):
        self.bit = 0
        self.left = None
        self.right = None
        self.leaf = False

##### Create a root node
root = Node()
##### Create a tree with root as root node
tree = Tree(root)
##### Build tree with height = logbase2 (n-ways) by calling build_tree function
tree.build_tree(root,4)

In [3]:
class Tree(object): 
    def __init__(self,root):
        self.root = root
        self.height = 0
    
    def insert(self, root, height):
        it = height-1
        if it == 0:
            root.right = []
            root.left = []
            root.leaf = True
            return
        root.left = Node()
        root.right = Node()
        self.insert(root.left,it)
        self.insert(root.right,it)
    
    def build_tree(self, root, ways):
        self.height = int(np.log2(ways))
        self.insert(root, self.height)
        return root

##### build cache args: cache_size in KB, no of words per line, no of ways
cache = CacheBuilder(4,2,4,8)
4 = 4KB of cache,   2 = 2 Words per cache line(8 bytes), 4 = no of ways, 8 = size of element to store in bytes 
##### create cache by calling create_cache() method
cache.create_cache()
##### access cache through the cache_dict dictionary. 
cache.cache_dict
The dictionary has all sets and each set has one tree. Each leaf node of tree is linked to two lines of the set

In [86]:
class CacheBuilder(object):
    # 1 word = 4bytes = 32bits
    # cache size in KB (kilobytes)
    # n = no of ways
    # line_size = no of words in one line 
    # i.e line_size is the max no of intigers that can be in one line 
    # assuming one int = 32 bits or 4 bytes or 1 word
    def __init__(self, cache_size, line_size, n, element_size_in_bytes):
        self.ways = n
        self.line_size = line_size
        self.cache_size = cache_size
        self.no_of_lines = (cache_size*1024)/(line_size*4) 
        self.element_size_in_bytes = element_size_in_bytes
        self.no_of_elements_per_line = self.line_size*4/(element_size_in_bytes) 
        if (n<=self.no_of_lines):
            self.no_of_sets = (self.no_of_lines/n)
        else:
            self.no_of_sets = 0
            print "no of ways cannot be less then total no of lines"
        self.cache_dict = {}
        
    def append_lines(self,lines,root):
        if root.leaf == True:
            lines.append(root.left)
            lines.append(root.right)
            return lines
        self.append_lines(lines,root.left)
        self.append_lines(lines,root.right)    
        return lines
    
    def create_cache(self):
        if(self.no_of_sets == 0):
            print "cache cannot be built, reason: no of sets cannot be less then no of lines"
            return
        # create a tree for each set
        for i in range(self.no_of_sets):
            self.cache_dict[i] = {}
            root = Node()
            tree = Tree(root)
            tree.build_tree(root, self.ways)
            self.cache_dict[i]["tree"] = tree
            #create lines
            lines = []
            self.cache_dict[i]["lines"] = self.append_lines(lines,tree.root)
            self.cache_dict[i]["tag"] = []
            for j in range(self.ways):
                self.cache_dict[i]["tag"].append("")
            
    def lookup(self, set_no, tag):
        if tag in self.cache_dict[set_no]["tag"]:
            return self.cache_dict[set_no]["lines"][self.cache_dict[set_no]["tag"].index(tag)]
            #return True
        else:
            return None
    
    def on_miss_load(self, element, set_no, tag):
        # load the element in the memory by PLRU algorith
        rt = self.cache_dict[set_no]["tree"].root
        while rt.leaf == False:
            if rt.bit == 0:
                rt.bit = 1
                rt = rt.left
            else:
                rt.bit = 0
                rt = rt.right
        if rt.bit == 0:
            rt.bit = 1
            if len(rt.left) > 0:
                rt.left[:] = []
            for num in element:
                rt.left.append(num)
        else:
            rt.bit = 0
            if len(rt.right) > 0:
                rt.right[:] = []
            for num in element:
                rt.right.append(num)
        # updata the tag field for that loaded element
        self.cache_dict[set_no]["tag"][self.cache_dict[set_no]["lines"].index(element)] = tag
        

##### create a Memory block 
mem = MemBuilder(MS, cache.line_size, cache.no_of_sets, cache.ways, cache.element_size_in_bytes)
##### multiply the two matrices
res = mem.multiply(cache)

In [114]:
class MemBuilder(object):
    # build the memory
    # memory is a 2d array column = line_size
    # rows = MS/line_size
    # line size is no of elements per memory line....one mem line = 64bits or 8bytes considering 64 bit arch 
    # no_of_sets = no of sets in 
    def __init__(self, MS, line_size, no_of_sets, no_of_ways, element_size):
        self.mat_size = MS
        self.line_size = line_size
        self.no_of_sets = no_of_sets
        self.element_per_mem_block = line_size*4/element_size
        #self.element_per_cache_line = line_size*4/element_size
        self.mem = []
        self.mat1 = []
        self.mat2 = []
        self.hit = 0
        self.miss = 0
        self.add_lookup = {}
        counter = 1
        counter_1 = 0
        # fill the memory here
        for i in range(((2*MS*MS)/self.element_per_mem_block)):
            line = []
            for j in range(self.element_per_mem_block):
                self.add_lookup[counter] = {}
                self.add_lookup[counter]["block_add"] = '{:032b}'.format(i)
                self.add_lookup[counter]["set"], self.add_lookup[counter]["tag"] = self.convert(
                    self.add_lookup[counter]["block_add"]) 
                line.append(counter)
                if counter <= (MS*MS):
                    self.mat1.append(counter)
                else:
                    self.mat2.append(counter)
                counter+=1
                counter_1+=(element_size*8)
            self.mem.append(line)
        # build two matrices of size MS*MS
        arr1 = np.array(self.mat1).reshape(MS,MS)
        arr2 = np.array(self.mat2).reshape(MS,MS)
        self.mat1 = arr1.tolist()
        self.mat2 = arr2.tolist()
        
    def convert (self, add):
        set_bits = np.log2(self.no_of_sets)
        tag_bits = 32-set_bits
        set_bin = add[-int(set_bits):]
        tag_bin = add[0:-int(set_bits)]
        return int(set_bin,2), int(tag_bin,2)
                
    # multiply both the matrix
    def multiply(self,cache):
        res = []
        for i in range (self.mat_size):
            res_row = []
            for j in range (self.mat_size):
                res_sum = 0
                for x in range(self.mat_size):
                    
                    # check if cache has the elements
                    set_no = self.add_lookup[self.mat1[i][x]]["set"]
                    tag = self.add_lookup[self.mat1[i][x]]["tag"]
                    lookup = cache.lookup(set_no, tag)
                    if lookup != None:
                        #print "was looking for ", self.mat1[i][x], "found in ", lookup
                        self.hit += 1
                    else:
                        #print "was looking for", self.mat1[i][x], "could not find in cache"
                        cache.on_miss_load(self.mem[(tag*self.no_of_sets)+set_no], set_no, tag)
                        self.miss +=1
                    set_no = self.add_lookup[self.mat2[i][x]]["set"]
                    tag = self.add_lookup[self.mat2[i][x]]["tag"]
                    lookup = cache.lookup(set_no, tag)
                    if lookup != None:
                        #print "was looking for ", self.mat2[i][x], "found in ", lookup
                        self.hit += 1
                    else:
                        self.miss +=1
                        #print "was looking for", self.mat2[i][x], "could not find in cache"
                        cache.on_miss_load(self.mem[(tag*self.no_of_sets)+set_no], set_no, tag)
                    # resume operation
                    
                    res_sum = res_sum + (self.mat1[i][x]*self.mat2[x][j])
                res_row.append(res_sum)
            res.append(res_row)
        return res

In [128]:
## create a cache of 4KB, 16 words(64 bytes) per line, 4 ways 
## and element size of 8 bytes(64bits, floating point is 64 bits)
cache = CacheBuilder(4,16,4,8)
cache.create_cache()

In [129]:
# see content of cache when it is empty
cache.cache_dict

{0: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7efeb3a8da90>},
 1: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7efeb3a8de50>},
 2: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7efeb3a8d990>},
 3: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7efeb3a8dad0>},
 4: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7efeb3a8dd10>},
 5: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7efeb814a090>},
 6: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7efeb814a190>},
 7: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7efeb814a290>},
 8: {'lines': [[], [], [], []],
  'tag': ['', '', '', ''],
  'tree': <__main__.Tree at 0x7efeb814a390>},
 9: {'lines': [[], [], [], []],
  'tag': ['', '', '', '

In [130]:
# build memory with matrix size, cache line size(block size in Words), no of sets in cache, no of ways in cache
# element size in bytes (8 for floating point, 4 for integer)
mem = MemBuilder(100, cache.line_size, cache.no_of_sets, cache.ways, cache.element_size_in_bytes)

In [131]:
# seememory arranged in blocks of 8 elements each
# reason : we have 16 words (64 bytes) per line of cache, and 8 bytes per element, 
# so 8 elements per block(line)
mem.mem

[[1, 2, 3, 4, 5, 6, 7, 8],
 [9, 10, 11, 12, 13, 14, 15, 16],
 [17, 18, 19, 20, 21, 22, 23, 24],
 [25, 26, 27, 28, 29, 30, 31, 32],
 [33, 34, 35, 36, 37, 38, 39, 40],
 [41, 42, 43, 44, 45, 46, 47, 48],
 [49, 50, 51, 52, 53, 54, 55, 56],
 [57, 58, 59, 60, 61, 62, 63, 64],
 [65, 66, 67, 68, 69, 70, 71, 72],
 [73, 74, 75, 76, 77, 78, 79, 80],
 [81, 82, 83, 84, 85, 86, 87, 88],
 [89, 90, 91, 92, 93, 94, 95, 96],
 [97, 98, 99, 100, 101, 102, 103, 104],
 [105, 106, 107, 108, 109, 110, 111, 112],
 [113, 114, 115, 116, 117, 118, 119, 120],
 [121, 122, 123, 124, 125, 126, 127, 128],
 [129, 130, 131, 132, 133, 134, 135, 136],
 [137, 138, 139, 140, 141, 142, 143, 144],
 [145, 146, 147, 148, 149, 150, 151, 152],
 [153, 154, 155, 156, 157, 158, 159, 160],
 [161, 162, 163, 164, 165, 166, 167, 168],
 [169, 170, 171, 172, 173, 174, 175, 176],
 [177, 178, 179, 180, 181, 182, 183, 184],
 [185, 186, 187, 188, 189, 190, 191, 192],
 [193, 194, 195, 196, 197, 198, 199, 200],
 [201, 202, 203, 204, 205, 206, 2

In [132]:
# see the block addresses, associated sets and tags of each element
mem.add_lookup

{1: {'block_add': '00000000000000000000000000000000', 'set': 0, 'tag': 0},
 2: {'block_add': '00000000000000000000000000000000', 'set': 0, 'tag': 0},
 3: {'block_add': '00000000000000000000000000000000', 'set': 0, 'tag': 0},
 4: {'block_add': '00000000000000000000000000000000', 'set': 0, 'tag': 0},
 5: {'block_add': '00000000000000000000000000000000', 'set': 0, 'tag': 0},
 6: {'block_add': '00000000000000000000000000000000', 'set': 0, 'tag': 0},
 7: {'block_add': '00000000000000000000000000000000', 'set': 0, 'tag': 0},
 8: {'block_add': '00000000000000000000000000000000', 'set': 0, 'tag': 0},
 9: {'block_add': '00000000000000000000000000000001', 'set': 1, 'tag': 0},
 10: {'block_add': '00000000000000000000000000000001', 'set': 1, 'tag': 0},
 11: {'block_add': '00000000000000000000000000000001', 'set': 1, 'tag': 0},
 12: {'block_add': '00000000000000000000000000000001', 'set': 1, 'tag': 0},
 13: {'block_add': '00000000000000000000000000000001', 'set': 1, 'tag': 0},
 14: {'block_add': '0

In [133]:
# do the operation
res = mem.multiply(cache)

In [134]:
# no of hits and misses
print "hits:",mem.hit
print "misses:", mem.miss

hits: 1997500
misses: 2500


In [135]:
# cache content at the end of the operation
cache.cache_dict

{0: {'lines': [[9985, 9986, 9987, 9988, 9989, 9990, 9991, 9992],
   [9857, 9858, 9859, 9860, 9861, 9862, 9863, 9864],
   [19841, 19842, 19843, 19844, 19845, 19846, 19847, 19848],
   [19969, 19970, 19971, 19972, 19973, 19974, 19975, 19976]],
  'tag': [78, 77, 155, 156],
  'tree': <__main__.Tree at 0x7efeb3a8da90>},
 1: {'lines': [[9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000],
   [9865, 9866, 9867, 9868, 9869, 9870, 9871, 9872],
   [19849, 19850, 19851, 19852, 19853, 19854, 19855, 19856],
   [19977, 19978, 19979, 19980, 19981, 19982, 19983, 19984]],
  'tag': [78, 77, 155, 156],
  'tree': <__main__.Tree at 0x7efeb3a8de50>},
 2: {'lines': [[19985, 19986, 19987, 19988, 19989, 19990, 19991, 19992],
   [19857, 19858, 19859, 19860, 19861, 19862, 19863, 19864],
   [9745, 9746, 9747, 9748, 9749, 9750, 9751, 9752],
   [9873, 9874, 9875, 9876, 9877, 9878, 9879, 9880]],
  'tag': [156, 155, 76, 77],
  'tree': <__main__.Tree at 0x7efeb3a8d990>},
 3: {'lines': [[19993, 19994, 19995, 19996, 19997, 